In [ ]:
# 1. 뉴스 기사 크롤링
# 크롤링하기 위한 모듈 import
from bs4 import BeautifulSoup

# URL에 접근하기 위한 모듈 import
import requests

In [ ]:
# TODO: 뉴스 검색할 키워드 입력
searchKeyword = "xxx"

# 뉴스 검색 결과를 가진 base URL
baseURL = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query={}'.format(searchKeyword)

# 해당 URL로 이동해서 HTML 내용 스크래핑
req = requests.get(baseURL)
 
if req.ok:
    html = req.text
    soup = BeautifulSoup(html,'html.parser')

In [ ]:
# 검색 결과로 나온 뉴스의 URL을 담을 리스트
newsURLs = []

# TODO: 스크래핑했던 HTML에서 URL만 추출 
urlList = soup.select('xxx')

# 네이버 뉴스에 해당하는 최종 URL 추출해서 뉴스 URL 리스트에 저장
for url in urlList:
    newsURL = url.get('href')
    if newsURL.startswith("https://news.naver.com"): 
        newsURLs.append(newsURL)
        
# 추출한 뉴스 URL 리스트 출력
print(newsURLs)

In [ ]:
# 각 뉴스 기사의 정보를 저장할 리스트
newsFinalInfo = []

# 각 뉴스 기사에서 날짜, 언론사, 뉴스 제목, 기사 내용 수집
for newsURL in newsURLs:
    # 각 기사의 디테일한 내용을 담을 리스트
    newsDetails = []
    
    # 해당 URL로 이동해서 해당 기사의 HTML 내용 스크래핑
    req = requests.get(newsURL)

    if req.ok:
        html = req.text
        soup = BeautifulSoup(html,'html.parser')
    
    # 기사 날짜 추출
    try:
        time = soup.select('.t11')[0].text
    except IndexError: 
        continue
    newsDetails.append(time)
    
    # 언론사 추출
    press = soup.select('.article_header > div > a > img')[0].get('title')
    newsDetails.append(press)
    
    # TODO: 기사 제목 추출
    title = soup.select('xxx')[0].text
    newsDetails.append(title)
    
    # 기사 내용 추출
    fullArticle = soup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    parsedArticle = fullArticle.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    newsDetails.append(parsedArticle.strip())
    
    # 기사 URL을 마지막 원소로 저장
    newsDetails.append(newsURL)
    
    # 위의 4가지 정보를 모아서 하나의 리스트로 만든 뒤, 최종 정보로 저장
    newsFinalInfo.append(newsDetails)

print(newsFinalInfo) 

In [ ]:
# 2. 크롤링한 뉴스 기사 내용들을 엑셀로 저장하기
# 데이터를 엑셀로 저장하고 조작하기 위한 Pandas 모듈 import
import pandas as pd

# 텍스트가 잘리는 현상 방지
pd.set_option('display.max_colwidth', -1)

# 위에서 수집한 뉴스 정보를 행렬 형태로 변환
data = pd.DataFrame(newsFinalInfo) 
data.columns = ['날짜', '언론사', '제목', '내용', 'URL']

# 상위 n행 출력
#data.head()

In [ ]:
# XlsxWriter 엔진을 사용해 엑셀 writer를 만들고, 엑셀 파일에 쓸 준비
writer = pd.ExcelWriter('news.xlsx', engine='xlsxwriter')
 
# 행렬 데이터를 엑셀 파일에 쓰기
data.to_excel(writer, sheet_name='Sheet1')

# 엑셀 writer를 닫아줌으로써 엑셀 쓰기 작업 종료
writer.close()

In [ ]:
# 3. 위에서 만든 dataframe 및 엑셀 파일을 이메일로 보내기
# 이메일을 보내는 데에 필요한 library import
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# TODO: 발신자 Gmail 주소/password 및 수신자 Gmail 주소 입력
sender = 'xxx@gmail.com'
password = 'xxx'
recipients = ['xxx@gmail.com']

# 메일 제목 입력
subject = searchKeyword + ' 관련 뉴스 크롤링 결과 공유 드립니다.' 

# 위의 메일 정보 저장
msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = ", ".join(recipients)
msg['Subject'] = subject

# TODO: 본문 텍스트 내용 입력
body = """
안녕하세요. xxx의 xxx입니다.

뉴스 크롤링 결과 아래 공유 드립니다.
엑셀 파일도 따로 첨부했으니 확인 부탁드립니다.

감사합니다.

"""
msg.attach(MIMEText(body, 'plain'))

# 위의 뉴스 결과를 HTML 표 형식으로 첨부
html = data.to_html()
msg.attach(MIMEText(html, 'html'))

In [ ]:
# 첨부파일 경로 및 이름 지정하기
fileName = 'news.xlsx'  
attachment = open(fileName, 'rb')

# 첨부파일 첨부
part = MIMEBase('application','octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= " + fileName)
msg.attach(part)

In [ ]:
# 이메일 전송 서버 준비
server = smtplib.SMTP('smtp.gmail.com',587)
server.starttls()

# 로그인 및 이메일 전송
server.login(sender, password)
server.sendmail(sender, recipient, msg.as_string())

# 이메일 전송 서버 종료
server.quit()

In [ ]:
# 4. 엑셀 파일 읽어와서 그 내용을 이메일로 보내기
# 엑셀 파일 읽어오기
readData = pd.read_excel('example.xlsx')

# 엑셀 파일 내용을 Data frame으로 저장
data = pd.DataFrame(readData)

# 읽어온 데이터 확인
print(data)

# 첫번째 열만 추출해서 다시 dataframe 형태로 저장
data = data[data.columns[0]].to_frame()

In [ ]:
# 첫번째 열 내용을 이메일로 전송
# 위의 메일 정보 다시 저장
msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = ", ".join(recipients)

# 메일 제목 입력
subject = '카페 메뉴 공유 드립니다.' 
msg['Subject'] = subject

# TODO: 본문 텍스트 내용 입력
body = """
안녕하세요. xxx의 xxx입니다.

저희 카페에서 판매하는 음료 종류는 아래와 같습니다.
확인 부탁드립니다.

감사합니다.

"""
msg.attach(MIMEText(body, 'plain'))

# 위의 뉴스 결과를 HTML 표 형식으로 첨부
html = data.to_html()
msg.attach(MIMEText(html, 'html'))

In [ ]:
# 이메일 전송 서버 준비
server = smtplib.SMTP('smtp.gmail.com',587)
server.starttls()

# 로그인 및 이메일 전송
server.login(sender, password)
server.sendmail(sender, recipient, msg.as_string())

# 이메일 전송 서버 종료
server.quit()